<a href="https://colab.research.google.com/github/Jtwulf/kthr_lab/blob/main/omnizart_drum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Environment Setup

!git clone https://github.com/Music-and-Culture-Technology-Lab/omnizart.git
%cd omnizart
!pip install -U pip
!pip install .
!omnizart download-checkpoints
!apt install fluidsynth
!pip install pyfluidsynth
!curl -L https://github.com/yt-dlp/yt-dlp/releases/latest/download/yt-dlp -o /usr/local/bin/yt-dlp
!chmod a+rx /usr/local/bin/yt-dlp

Cloning into 'omnizart'...
remote: Enumerating objects: 4361, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 4361 (delta 44), reused 33 (delta 22), pack-reused 4275
Receiving objects: 100% (4361/4361), 74.34 MiB | 18.47 MiB/s, done.
Resolving deltas: 100% (2945/2945), done.
/content/omnizart
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/omnizart
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8

In [ ]:
!pip install -U pip
!pip install git+https://github.com/Music-and-Culture-Technology-Lab/omnizart
!omnizart download-checkpoints
!apt install fluidsynth
!pip install pyfluidsynth
!curl -L https://github.com/yt-dlp/yt-dlp/releases/latest/download/yt-dlp -o /usr/local/bin/yt-dlp
!chmod a+rx /usr/local/bin/yt-dlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/Music-and-Culture-Technology-Lab/omnizart to /tmp/pip-req-build-vqjqujdn
  Running command git clone --filter=blob:none --quiet https://github.com/Music-and-Culture-Technology-Lab/omnizart /tmp/pip-req-build-vqjqujdn
  Resolved https://github.com/Music-and-Culture-Technology-Lab/omnizart to commit a2f4c614dbac06c8361e40e0d71ae5e24da638af
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
  Using cached jsonschema-3.2.0-py2.py3-none-any.whl (56 kB)
  Using cached librosa-0.8.1-py3-none-any.whl (203 kB)
  Using cached madmom-0.16.1.tar.gz (20.0 MB)
  Preparing metadata (setup.py) ... done
  Using cached mir_eval-0.6.tar.gz (87 

In [ ]:
#@title Upload MP3 File
import os
from google.colab import files
from IPython import display as dsp

ff = files.upload()
uploaded_audio = list(ff.keys())[0].replace(".mp3", "")

!test -f "$uploaded_audio".wav && rm "$uploaded_audio".wav
!ffmpeg -i "$uploaded_audio".mp3 "$uploaded_audio".wav &>/dev/null

dsp.Audio(uploaded_audio + ".mp3") if os.path.exists(uploaded_audio + ".mp3") else None

MessageError: ignored

In [ ]:
#@title Transcribe

mode = "drum" #@param ["music-piano", "music-piano-v2", "music-assemble", "chord", "drum", "vocal", "vocal-contour", "beat"]

model = ""
if mode.startswith("music"):
  mode_list = mode.split("-")
  mode = mode_list[0]
  model = "-".join(mode_list[1:])


from omnizart.music import app as mapp
from omnizart.chord import app as capp
from omnizart.drum import app as dapp
from omnizart.vocal import app as vapp
from omnizart.vocal_contour import app as vcapp
from omnizart.beat import app as bapp

app = {
    "music": mapp,
    "chord": capp,
    "drum": dapp,
    "vocal": vapp,
    "vocal-contour": vcapp,
    "beat": bapp
}[mode]

model_path = {
    "piano": "Piano",
    "piano-v2": "PianoV2",
    "assemble": "Stream",
    "pop-song": "Pop",
    "": None
}[model]

midi = app.transcribe(f"{uploaded_audio}.wav", model_path=model_path)

# Synthesize MIDI and play
import scipy.io.wavfile as wave
from omnizart.remote import download_large_file_from_google_drive

SF2_FILE = "general_soundfont.sf2"
if not os.path.exists(SF2_FILE):
  print("Downloading soundfont...")
  download_large_file_from_google_drive(
      "16RM-dWKcNtjpBoo7DFSONpplPEg5ruvO",
      file_length=31277462,
      save_name=SF2_FILE
    )

if mode == "vocal-contour":
  os.rename(f"{uploaded_audio}_trans.wav", f"{uploaded_audio}_synth.wav")
else:
  print("Synthesizing MIDI...")
  out_name = f"{uploaded_audio}_synth.wav"
  raw_wav = midi.fluidsynth(fs=44100, sf2_path=SF2_FILE)
  wave.write(out_name, 44100, raw_wav)

!ffmpeg -i "$out_name" "tmp_synth.mp3" &>/dev/null
!mv tmp_synth.mp3 "$uploaded_audio"_synth.mp3

out_name = out_name.replace(".wav", ".mp3")
print(f"Finished: {out_name}")
dsp.Audio(out_name)

FileNotFoundError: ignored

In [ ]:
#@title Download MIDI

files.download(f"{uploaded_audio}.mid")

In [ ]:
#@title Download MP3

files.download(out_name)